**Universidad de Los Andes - Facultad de Economía** <br>
**Machine Learning para Business Intelligence** <br>
**_Paula Rodríguez, Juan S. Moreno, Mateo Dulce_**
# Clase 3: Modelos de Regresión

En esta clase se cubriran los modelos estándar de regresión:

- Mínimos cuadrados ordinarios (Regresión Lineal)
- Regresión polinomial
- Regresión Logística
- Linear Discriminant Analysis
- Regularización L1 y L2
- Árboles de regresión

Además, para evaluar el desempeño de los modelos implementados, se estudiarán las métricas de evaluación:
- R-cuadrado
- MSE
- MAPE

<img src="https://raw.githubusercontent.com/RodrigoLaraMolina/DPATTSrc/master/ML2.png" alt="ML" style="width: 500px;" align="center" frameborder="200"/>

_(Imagen tomada de [Medium - Different types of Machine learning and their types.](https://medium.com/deep-math-machine-learning-ai/different-types-of-machine-learning-and-their-types-34760b9128a2))_


## 0. Introducción a Scikit-Learn

Existen varias librerias de Python que proveen implementaciones solidas de multiples algoritmos de aprendizaje de máquinas. Una de las más conocidas es [Scikit-Learn](http://scikit-learn.org). Scikit-Learn se caracteriza por ser una API limpia, uniforme y optimizada, y por tener una documentación en línea muy útil y completa.Una ventaja de esta uniformidad es que una vez que comprende el uso básico y la sintaxis de Scikit-Learn para un tipo de modelo, cambiar a un nuevo modelo o algoritmo es muy sencillo.

### Uso básico de la API

Por lo general, los pasos a seguir para utilizar los modelos implementados en Scikit-Learn son:

1. Seleccionar una clase de modelo importando la clase de estimador apropiada de Scikit-Learn.
2. Seleccionar los hiper parámetros del modelo al instanciar la clase anterior con estos valores.
3. Organizar los datos en una matriz de variables y un vector objetivo.
4. Ajustar el modelo a los datos llamando al método `` fit () `` de la instancia del modelo.
5. Evaluar el modelo en nuevos datos.

## 1. Mínimos Cuadrados Ordinarios
### Regresión lineal univariada

Los modelos de regresión lineal son un buen punto de partida para las tareas de regresión. Dichos modelos son populares porque pueden ajustarse muy rápidamente y son muy interpretables.

Comenzaremos con la regresión lineal más familiar, un ajuste en línea recta a los datos.
Un ajuste en línea recta es un modelo de la forma
$$
y = ax + b
$$
donde $a$ es la *pendiente*, y $b$ is el *intercepto*.

Considere los siguientes datos, que están dispersos alrededor de un modelo lineal con una pendiente de 2 y una intersección de -5:

In [ ]:
# importar paquetes estandar
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

### Generar datos sintéticos

In [ ]:
# para tener replicabilidad
rng = np.random.RandomState(1)

# generamos puntos distribuidos de manera uniforme entre 0 y 10
x = 10 * rng.rand(50,1)

# generamos los valores de y usando un modelo lineal con pendiente 2
# e intercepto -5 mas un ruido alatorio
y = 2 * x[:,0] - 5 + rng.randn(50)

# construit un grafico de puntos
plt.scatter(x, y);

### Encontrar el mejor ajuste (usando Numpy)

Escribiendo el modelo de forma matricial tenemos: $Y = X^T \beta$.

Queremos encontrar $\hat \beta$ que minimice el costo $(Y-\hat Y)^2$ donde $\hat Y = X^T \hat \beta$. <br>
Esto es $\hat \beta = (X^TX)^{-1}X^TY$

In [ ]:
# resolvamos el problema utilziando el paquete numpy
import numpy as np

print(type(x),type(y))

In [ ]:
# creamos la matriz que contiene una columna de 1's y la variable x
X = np.matrix([np.ones(len(x),dtype=x.dtype),x.reshape(-1)])

# np.matrix toma cada entrada como una fila.
# trasponemos para tener observaciones x variables
X=X.T

In [ ]:
# estimación de beta
beta_estimado = (np.linalg.inv(X.T*X)*X.T).dot(y)
print("Pendiente del modelo: {}".format(beta_estimado[0,1]))
print("Intercepto del modelo: {}".format(beta_estimado[0,0]))

### Encontrar el mejor ajuste utilizando Scikit-Learn

In [ ]:
#importar LinearRegression de Scikit-Learn
from sklearn.linear_model import LinearRegression

# crear un modelo lineal con intercepto
model = LinearRegression(fit_intercept=True)

# ajustar el modelo a los datos
model.fit(x, y)

# graficar la funcion lineal estimada
xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

# build plot
plt.scatter(x, y)
plt.plot(xfit, yfit, color='black');

In [ ]:
print("Pendiente del modelo:    ", model.coef_[0])
print("Intercepto del modelo:", model.intercept_)

## Métricas de desempeño

- $MSE = mean((Y-\hat Y)^2)$ 
<br>
<br>
- $MAPE = mean(\mid Y-\hat Y \mid)$ 
<br>
<br>
- $R^2 = \dfrac{\sigma^{2}_{XY}}{\sigma^{2}_{X}\sigma^{2}_{Y}}$ donde $\sigma^{2}_{XY}$ es la covarianza de $X,Y$, $\sigma^{2}_{X}$ la varianza de $X$ y $\sigma^{2}_{Y}$ la varianza de $Y$. 

In [ ]:
# realizar la predicción dentro de muestra
y_pred = model.coef_[0]*x + model.intercept_

In [ ]:
# error cuadrático medio en el entrenamiento
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y,y_pred)
print("El modelo de regresión lineal tiene un MSE = {0:.3f} dentro de muestra".format(mse))

In [ ]:
# error cuadrático medio en el entrenamiento
from sklearn.metrics import mean_absolute_error
mape = mean_absolute_error(y,y_pred)
print("El modelo de regresión lineal tiene un MAPE = {0:.3f} dentro de muestra".format(mape))

In [ ]:
# error cuadrático medio en el entrenamiento
from sklearn.metrics import r2_score
r2 = r2_score(y,y_pred)
print("El modelo de regresión lineal tiene un R2 = {0:.3f} dentro de muestra".format(r2))

Existen otros paquetes que contienen implementaciones de este mismo modelo. Por ejemplo pueden pribar con: <br>
``statsmodels.formula.api.ols``

### Regresión Lineal Múltiple

El estimador ``LinearRegression`` también puede estimar modelos multivariables de la forma
$$
y = a_0 + a_1 x_1 + a_2 x_2 + \cdots
$$
donde hay multiples valores para $x$.
Geométricamente, esto es ajustar un plano a puntos en tres dimenciones o un hiper plano para dimensiones mayores.

In [ ]:
# creamos datos con 5 variables distribuidas uniformemente entre 0 y 10
# creamos la variable dependiente y como un modelo lineal de x
rng = np.random.RandomState(1) # replicabilidad
X = 10 * rng.rand(100, 2)
y = 0.5 + np.dot(X, [1.5, -2.])

In [ ]:
model.fit(X, y)
print(model.intercept_)
print(model.coef_)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(211,projection='3d')

ax.scatter(X.T[0],X.T[1], c='r', marker='o')

# generar una grilla de x,y
xx, yy = np.meshgrid(X.T[0],X.T[1])

exog = pd.core.frame.DataFrame({'X': xx.ravel(), 'Y': yy.ravel()})
out = model.predict(exog)
ax.plot_surface(xx, yy,
                out.reshape(xx.shape),
                rstride=1,
                cstride=1,
                color='None',
                alpha = 0.4)
plt.show()

## 2. Regresión polinomial

Una manera de adaptar la regresión lineal a una relación no lineal entre las variabels es transformar los datos de acuerda a *funciones base*.
La idea es tomar el modelo lineal multivariable:
$$
y = a_0 + a_1 x_1 + a_2 x_2 + a_3 x_3 + \cdots
$$
y construir $x_1, x_2, x_3,$ del input unidimensional $x$.
Tenemos entonces $x_n = f_n(x)$, donde $f_n()$ es una función que transforma nuestros datos.

Por ejemplo, si $f_n(x) = x^n$, nuestro modelo se convierte en una regresión polinomial:
$$
y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots
$$
Note que este modelo sigue siendo un *modelo lineal*. La linearidad se refiere al hecho de que los coeficientes $a_n$ son lineales entre ellos. Lo que hemos hecho es tomar la variable unidimensional $x$ y proyectarla en dimensiones mayores para que así el ajuste lineal pueda ajustar relaciones más complejas entre $x$ y $y$.

In [ ]:
# función de scikit learn que crea tranforma los datos
from sklearn.preprocessing import PolynomialFeatures 

x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

La manera más 'limpia' de realizar *feature engineering* en los modelos de ML es utilizando un pipeline. Creamos ahora un modelo polinomial de grado 7 usando ``make_pipeline`` y ``LinearRegression`` de Scikit Learn.

In [ ]:
# definimos el modelo polinomial de grado 7
from sklearn.pipeline import make_pipeline
poly_model = make_pipeline(PolynomialFeatures(7),
                           LinearRegression())

In [ ]:
rng = np.random.RandomState(1) #replicabilidad
x = 10 * rng.rand(50) # creacion de x aleatorio entre 0 y 10
y = np.sin(x) + 0.1 * rng.randn(50) # creacion de y como respuesta no lineal de x

poly_model.fit(x[:, np.newaxis], y) #ajuste del modelo polinomial
yfit = poly_model.predict(xfit[:, np.newaxis]) #prediccion de y

plt.scatter(x, y)
plt.plot(xfit, yfit, color='black');

In [ ]:
# graficar valores reales de y vs prediccion
y_pred = poly_model.predict(x[:, np.newaxis])
plt.scatter(y,y_pred)
plt.xlabel("Real: $Y_i$")
plt.ylabel("Predicted: $\hat{Y}_i$")
plt.title("Real vs Predicted: $Y_i$ vs $\hat{Y}_i$")
plt.plot([-1,1.2],[-1,1.2],color="black")
plt.show()

### Selección de grado con validación

Para esta y la siguiente sección estudiaremos el **Conjunto de datos de vivienda de Boston** que consiste en el precio de las casas en varios lugares de Boston. Junto con el precio, el conjunto de datos también proporciona información como la tasa de criminalidad (CRIM), áreas de negocios no minoristas en la ciudad (INDUS), la edad de las personas que poseen la casa (EDAD) y hay muchos otros atributos

In [ ]:
#5.2.1. Boston house prices dataset

# display an Inline Frame
from IPython.display import IFrame 
IFrame('https://scikit-learn.org/stable/datasets/index.html#boston-dataset',
       width=750, height=500)

In [ ]:
#load the Boston data set in it's original format (dictionary)
from sklearn.datasets import load_boston
raw_data = load_boston()

# Create a Pandas Data Frame with the Boston data
boston = pd.DataFrame(raw_data.data)
boston.columns = raw_data.feature_names
boston['PRICE'] = raw_data.target
display(boston.head(10))

In [ ]:
# definimos una función para el pipeline de la regresion polinomial
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [ ]:
# obtenemos datos univariados del dataset de boston
X = boston.LSTAT.values.reshape(-1, 1)
y = boston.PRICE.values.reshape(-1, 1)

In [ ]:
%matplotlib inline
import seaborn; seaborn.set()  # plot formatting

X_test = np.linspace(1, 40, 500)[:, None]

fig = plt.figure(figsize=(10,7))
plt.scatter(X.ravel(), y, color='black',s=5)
axis = plt.axis()
for degree in [1, 3, 7]:
    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
#plt.xlim(-0.1, 1.0)
#plt.ylim(-2, 12)
plt.legend(loc='best');

In [ ]:
from sklearn.model_selection import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          'polynomialfeatures__degree', degree, cv=7)

fig = plt.figure(figsize=(8,5))
plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

#### Validación con GridSearchCV

<img src="img/fold-CV.png" width="500">

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=5)

In [ ]:
# ajustamos el GridSearchCV a los datos
grid.fit(X, y);

In [ ]:
grid.best_params_

In [ ]:
model = grid.best_estimator_

fig = plt.figure(figsize=(8,5))
plt.scatter(X.ravel(), y, s=10)
lim = plt.axis()
y_test = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test, color='black');
plt.axis(lim);

## 3. Regularización Ridge y Lasso

La regresión de Ridge y Lasso son técnicas generalmente utilizadas para crear modelos que generealicen bien al tener una gran cantidad de variables. Aquí "grande" puede significar cualquiera de dos cosas:

- Lo suficientemente grande como para mejorar la tendencia de un modelo a sobreajustar
- Lo suficientemente grande como para causar desafíos computacionales

Modelos de Regularización:

1. **Ridge:** Realiza la regularización L2, es decir, agrega una penalización equivalente al cuadrado de la magnitud de los coeficientes
2. **Lasso:** Realiza la regularización L1, es decir, agrega una penalización equivalente al valor absoluto de la magnitud de los coeficientes



In [ ]:
rng = np.random.RandomState(8)
x = 10 * rng.rand(40)
y = np.sin(x) + 0.2 * rng.randn(40)

plt.scatter(x, y, s=15)
plt.show()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, test_size = 0.3, random_state = 8)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
    
    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))
        
    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
        
    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_,
                                 self.width_, axis=1)

In [ ]:
model = make_pipeline(GaussianFeatures(18),
                      LinearRegression())
model.fit(X_train[:, np.newaxis], Y_train)

plt.scatter(X_train, Y_train)
plt.scatter(X_test, Y_test, color='red')
plt.plot(np.linspace(0, 10, 1000), model.predict(xfit[:, np.newaxis]))
plt.title("Regresión sin regularización")
plt.show()

In [ ]:
mse_train = mean_squared_error(Y_train,model.predict(X_train[:, np.newaxis]))
mse_test = mean_squared_error(Y_test,model.predict(X_test[:, np.newaxis]))
print("El MSE en entrenamiento del modelo Rodge es: {0:.3f}".format(mse_train))
print("El MSE en prueba del modelo Rodge es: {0:.3f}".format(mse_test))

### Rregresión Ridge

In [ ]:
from sklearn.linear_model import Ridge
ridge = make_pipeline(GaussianFeatures(18), Ridge(alpha=0.01))
ridge.fit(X_train[:, np.newaxis],Y_train)

In [ ]:
plt.scatter(X_train, Y_train)
plt.scatter(X_test, Y_test, color='red')
plt.plot(np.linspace(0, 10, 1000), ridge.predict(xfit[:, np.newaxis]))
plt.title("Regresión con regularización Ridge")
plt.show()

In [ ]:
mse_train = mean_squared_error(Y_train,ridge.predict(X_train[:, np.newaxis]))
mse_test = mean_squared_error(Y_test,ridge.predict(X_test[:, np.newaxis]))
print("El MSE en entrenamiento del modelo Ridge es: {0:.3f}".format(mse_train))
print("El MSE en prueba del modelo Ridge es: {0:.3f}".format(mse_test))

### Regresión Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso = make_pipeline(GaussianFeatures(18), Lasso(alpha=0.01))
lasso.fit(X_train[:, np.newaxis],Y_train)

In [ ]:
plt.scatter(X_train, Y_train)
plt.scatter(X_test, Y_test, color='red')
plt.plot(np.linspace(0, 10, 1000), lasso.predict(xfit[:, np.newaxis]))
plt.title("Regresión con regularización Lasso")
plt.show()

In [ ]:
mse_train = mean_squared_error(Y_train,lasso.predict(X_train[:, np.newaxis]))
mse_test = mean_squared_error(Y_test,lasso.predict(X_test[:, np.newaxis]))
print("El MSE en entrenamiento del modelo Lasso es: {0:.3f}".format(mse_train))
print("El MSE en prueba del modelo Lasso es: {0:.3f}".format(mse_test))

## 4. Linear Discriminant Analysis

El _Linear Discriminant Analysis (LDA)_ es una técnica supervisada de reducción de dimensionalidad. El objetivo es proyectar un conjunto de datos en un espacio de dimensiones inferiores con buena capacidad de separación de clases para evitar el sobreajuste y también reducir los costos computacionales.

<img src="img/lda1.png" width="250" class="center"><img src="img/lda2.png" width="250" class="center"><img src="img/lda3.png" width="250" class="center"><img src="img/lda4.png" width="250" class="center">

In [ ]:
from sklearn.datasets import load_wine #importo datos
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis #importar LDA de scikit learn

In [ ]:
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Categorical.from_codes(wine.target, wine.target_names)

In [ ]:
X_lda = LinearDiscriminantAnalysis().fit_transform(X, y)
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=pd.factorize(y)[0] + 1,
    alpha=0.7,
    edgecolors='b'
)
plt.show()

## 5. Árboles de regresión

Cubriremos los siguientes temas:
- Selección de hiper-parámetros.
- Interpretación de los árboles.
- Importancia de variables.
- Tipos de ensambles: simulatáneos y secuenciales.
- Regularización de árboles.
- Evaluación de los modelos.

Estudiaremos los siguientes tres modelos:
- Árboles de regresión
- Random Forest (Bosques aleatorios)
- Gradient Boosting Machine

### Árboles de regresión

Los árboles de decisión son formas extremadamente intuitivas para clasificar o etiquetar objetos: simplemente hace una serie de preguntas diseñadas para concentrarse en la regresión o clasificación.
Por ejemplo, si desea construir un árbol de decisión para clasificar una casa, puede construir el que se muestra aquí:

<img src="https://raw.githubusercontent.com/RodrigoLaraMolina/DPATTSrc/master/decision-tree-house.png" alt="decision_tree" style="width: 700px;" align="center" frameborder="200"/>

<img src="img/regression_trees.png" width="800" class="center">

In [ ]:
# import tree from Scikit-Learn
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

In [ ]:
# Separamos las variables independientes y dependiente
X = boston.drop('PRICE', axis = 1)
Y = boston['PRICE']

# importar función para separar dataset en entrenamiento y prueba
from sklearn.model_selection import train_test_split

# Separamos dataset con test del 30% de las observaciones
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3, random_state = 5)

In [ ]:
# definimos el arbol de decisión
tree_model = DecisionTreeRegressor(random_state=5)

# ajustar el modelo a los datos
tree_model.fit(X_train, Y_train)

#### Visualización del árbol

In [ ]:
# importar los paquetes para visualizar arboles de decision
from graphviz import Source
from IPython.display import Image

# crear la visualizacion como png
graph = Source(tree.export_graphviz(tree_model, out_file=None,
                                     feature_names=X.columns))
png_bytes = graph.pipe(format='png')
with open('dtree_pipe.png','wb') as f:
    f.write(png_bytes)

# mostrar la imagen creada   
from IPython.display import Image
print('Double click to zoom')
Image(png_bytes)

In [ ]:
# error de entranamiento y prueba
pred_train = tree_model.predict(X_train)
pred_test= tree_model.predict(X_test)
mse_train = mean_squared_error(Y_train,pred_train)
mse_test = mean_squared_error(Y_test, pred_test)
print("El mse de entrenamiento es: {0:.3f}".format(mse_train))
print("El mse de prueba es: {0:.3f}".format(mse_test))

__HAY OVERFITTING!!!__

#### Regularización de árboles (_podar el árbol_)

In [ ]:
param_grid = {'max_depth': list(range(2,15)), 
              'max_leaf_nodes': [5,10,15,20,None]}

grid = GridSearchCV(DecisionTreeRegressor(random_state=5), param_grid, cv=5)
grid.fit(X_train, Y_train)
grid.best_params_

In [ ]:
best_tree = grid.best_estimator_

In [ ]:
# crear la visualizacion como png
graph = Source(tree.export_graphviz(best_tree, out_file=None,
                                     feature_names=X.columns))
png_bytes = graph.pipe(format='png')
with open('best_tree.png','wb') as f:
    f.write(png_bytes)

# mostrar la imagen creada   
from IPython.display import Image
print('Double click to zoom')
Image(png_bytes)

In [ ]:
# error de entranamiento y prueba
pred_train = best_tree.predict(X_train)
pred_test= best_tree.predict(X_test)
mse_train = mean_squared_error(Y_train,pred_train)
mse_test = mean_squared_error(Y_test, pred_test)
print("El mse de entrenamiento para el mejor árbol es: {0:.3f}".format(mse_train))
print("El mse de prueba para el mejor árbol es: {0:.3f}".format(mse_test))

#### Variables más importantes

In [ ]:
# Creamos un dataframe con las importancias
importancia_arbol=pd.DataFrame(best_tree.feature_importances_, index=X.columns, columns=['Importancia'])
importancia_arbol

### Random Forest y Gradient Boosting

In [ ]:
from sklearn.ensemble import RandomForestRegressor # Importo la función para estimar RandomForest
from sklearn.ensemble import GradientBoostingRegressor # Importo la función para estimar el boosting

In [ ]:
# Defino el RandomForest
rf = RandomForestRegressor(random_state=23)

# ajusto el modelos
rf.fit(X_train, Y_train)

In [ ]:
# Defino el GBM
gbm = GradientBoostingRegressor(random_state=23)

# ajusto el modelos
gbm.fit(X_train, Y_train)

In [ ]:
# error de entranamiento y prueba
pred_train_rf = rf.predict(X_train)
pred_test_rf= rf.predict(X_test)

pred_train_gbm = gbm.predict(X_train)
pred_test_gbm= gbm.predict(X_test)


print("El mse de entrenamiento para el RF es: {0:.3f}".format(mean_squared_error(Y_train,pred_train_rf)))
print("El mse de prueba para el RF es: {0:.3f}".format(mean_squared_error(Y_test,pred_test_rf)))
print(" ")
print("El mse de entrenamiento para el GBM es: {0:.3f}".format(mean_squared_error(Y_train,pred_train_gbm)))
print("El mse de prueba para el GBM es: {0:.3f}".format(mean_squared_error(Y_test,pred_test_gbm)))


_**Ejercicio para la casa:** Utilice ``GridSearchCV`` para seleccionar los parámetros que mejor ajusten los modelos de Random Forest y Gradient Boosting._

### Resumen Rápido: Qué hemos hecho hasta ahora?
_Imagen tomada de [Dataconomy: INFOGRAPHIC: A BEGINNER’S GUIDE TO MACHINE LEARNING ALGORITHMS](https://dataconomy.com/2017/03/beginners-guide-machine-learning/)_

<img src="img/resumen.png" width="800">



## 4. Regresión Logística

El objetivo de esta sección es usar datos del orden financiero para predecir la probabilidad de que una persona deje de pagar sus deudas (incurra en default) dadas sus características observadas. Los datos fueron obtenidos del repositorio de machine learning de la UCI y corresponden a información crediticia de un importante banco en Taiwan.

Las variables corresponden a: 

- ID: Variable de identificación del cliente.
- LIMIT_BAL: Valor del crédito en dólares NT.
- SEX: Género de la persona (1= hombre, 2= mujer).
- EDUCATION: Nivel educativo (1= posgrado, 2= pregrado, 3=secundaria, 4=otros)
- MARRIAGE: Estado civil (1= casado, 2=soltero, 3= otro)
- AGE= Edad en años.
- PAY_0-PAY_6 = Historial de pagos pasados (de abril a septiembre). La escala de los estados son: -1= a tiempo, 1= pago con retraso de un mes, 2=  pago con retraso de dos meses, .....9= pago con retraso de nueve meses o más.
- BILL_AMT1- BILL_AMT6=  Valor a pagar (factura) en dólar NT (de abril a septiembre).
- PAY_AMT1-PAY_AMT6: Valor efectivamente pagado en dólar NT (de abril a septiembre).

### Cargar librerías

In [ ]:
import pandas as pd # Procesamiento de datos
import os  # Cambiar directorio de trabajo
import numpy as np # Arreglos númericos
from matplotlib import pyplot as plt # Gráficos

from sklearn.compose import ColumnTransformer  # Utilidades de  Preprocesamiento
from sklearn.pipeline import Pipeline          # Utilidades de  Preprocesamiento
from sklearn.preprocessing import StandardScaler, OneHotEncoder #Estandarización y dummies
from sklearn.linear_model import LogisticRegression  # Regresión logística
from sklearn.model_selection import train_test_split, GridSearchCV # Partición de datos, validación cruzada
import  sklearn.metrics as metrics  # Métricas de desempeño

### Importar los datos

In [ ]:
# Importar datos
dt= pd.read_csv("default.csv")
dt.head()

In [ ]:
#Seleccionar variables categóricas
catvars= list(["SEX","EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]) 

# Convertir variables categóricas
for col in catvars:
    dt[col] = dt[col].astype('category')

# Mostrar tipos de variables
dt.dtypes

In [ ]:
# Seleccionar características y marca
X = dt.drop(['default'], axis=1)
y = dt['default']

#convertir variables categoricas en dummies
X = pd.get_dummies(X, columns=catvars)

# Particionar los datos  70% entrenamiento, 30% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [ ]:
numvars

In [ ]:
# Estandarizar train y test
# Seleccionar variables númericas
numvars=X.select_dtypes(include=[np.number]).columns.tolist()

X_train.loc[:, numvars] = StandardScaler().fit_transform(X_train.loc[:, numvars])
X_test.loc[:, numvars] = StandardScaler().fit_transform(X_test.loc[:, numvars])

### Regresión Logística

In [ ]:
np.random.seed(888)

# Regresión logística
logit = LogisticRegression(solver="lbfgs", max_iter=2000, C=1000000)

# Ajustar Modelo
logit.fit(X_train, y_train)

# Predecir
pred_train = logit.predict_proba(X_train)
pred_test = logit.predict_proba(X_test)

# Calcular MSE
mse_train = mean_squared_error(y_train.values, pred_train)
mse_test = mean_squared_error(y_test.values, pred_test)